In [1]:
data_var = '2024-01-23'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10332,2024-01-23,Brasil Nbb,20:00,Flamengo,Caxias do Sul,1.09,6.41,156.5,1.84,1.86,-15.5,2.06,1.68,pU7OgVNO,0.917431,0.156006,0.543478,0.537634,0.073437,0.6,0.6,NaN,NaN,103.260,19.560978,0.189434,270.672,160.041166,0.591273,116.170,231.646,99.12,269.37,0.0,0.0,0.0,0.0,1.003149,0.007644,0.143690,1.51,0.302,0.298013,0.000000,0.0,0.000000,-2.80,-0.560,-9.660714,0.306392,0.3,-0.006392
10333,2024-01-23,Brasil Nbb,20:00,São Paulo,Paulistano,2.09,1.69,153.5,1.84,1.86,1.5,1.94,1.77,bXe6IE0P,0.478469,0.591716,0.543478,0.537634,0.070185,0.4,0.4,NaN,NaN,168.022,66.183280,0.393897,120.432,31.384923,0.260603,134.934,104.948,231.84,119.85,0.0,0.0,0.0,0.0,0.149652,0.007644,0.064802,-3.30,-0.660,-1.651515,0.636269,0.4,-0.236269,1.78,0.356,1.938202,0.000000,0.0,0.000000
10334,2024-01-23,Eua Nba,00:00,Sacramento Kings,Atlanta Hawks,1.33,3.77,236.5,1.83,2.05,-8.5,2.06,1.87,rcmRJaTF,0.751880,0.265252,0.546448,0.487805,0.017132,0.6,0.4,NaN,NaN,245.488,106.575396,0.434137,200.876,94.469352,0.470287,172.140,259.094,157.30,365.15,0.0,0.0,0.0,0.0,0.676604,0.080187,0.068372,-1.90,-0.380,-0.868421,0.709714,0.5,-0.209714,-0.25,-0.050,-55.400000,0.444599,0.4,-0.044599
10335,2024-01-23,Eua Nba,21:00,Indiana Pacers,Denver Nuggets,2.37,1.68,240.5,1.89,2.03,2.5,2.02,1.90,xKnVIJrM,0.421941,0.595238,0.529101,0.492611,0.017179,0.6,0.4,NaN,NaN,376.978,172.608369,0.457874,192.148,62.264140,0.324043,236.552,201.136,151.20,126.56,0.0,0.0,0.0,0.0,0.240940,0.050508,0.043292,1.87,0.374,3.663102,0.625849,0.7,0.074151,0.36,0.072,9.444444,0.642828,0.8,0.157172
10336,2024-01-23,Eua Nba,21:30,Brooklyn Nets,New York Knicks,2.56,1.60,223.5,1.81,2.00,2.5,2.10,1.74,GAoZHwbS,0.390625,0.625000,0.552486,0.500000,0.015625,0.8,0.2,NaN,NaN,292.716,151.108922,0.516230,141.724,15.626173,0.110258,229.426,242.434,171.95,148.40,0.0,0.0,0.0,0.0,0.326357,0.070525,0.132583,-0.84,-0.168,-9.285714,0.584931,0.4,-0.184931,0.86,0.172,3.488372,0.474315,0.6,0.125685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10398,2024-01-23,Rússia Liga Vtb United,13:30,MBA Moscow,Unics Kazan,5.15,1.17,157.5,1.83,1.93,9.5,2.07,1.70,ILrBok65,0.194175,0.854701,0.546448,0.518135,0.048876,0.4,0.8,NaN,NaN,182.492,91.146996,0.499457,93.768,13.542146,0.144422,160.780,104.044,148.80,106.40,0.0,0.0,0.0,0.0,0.890597,0.037612,0.138795,-3.40,-0.680,-6.102941,0.000000,0.0,0.000000,-0.15,-0.030,-5.666667,0.000000,0.0,0.000000
10399,2024-01-23,Uruguai Liga Uruguaia,20:00,Defensor Sporting,Malvin,1.50,2.47,160.5,1.87,1.79,-5.5,2.06,1.68,t0cRgNG7,0.666667,0.404858,0.534759,0.558659,0.071525,0.8,0.6,NaN,NaN,165.292,67.708386,0.409629,152.484,36.818276,0.241457,143.702,174.836,163.20,214.02,0.0,0.0,0.0,0.0,0.345538,0.030912,0.143690,-1.86,-0.372,-1.344086,0.000000,0.0,0.000000,-5.00,-1.000,-1.470000,0.000000,0.0,0.000000
10400,2024-01-23,Uruguai Liga Uruguaia,20:00,Hebraica y Macabi,Larre Borges,1.06,8.10,165.5,1.80,1.86,-16.5,2.00,1.72,th32I6g8,0.943396,0.123457,0.555556,0.537634,0.066853,0.8,0.2,NaN,NaN,138.346,22.016496,0.159141,382.636,94.967723,0.248193,143.848,334.342,119.34,247.04,0.0,0.0,0.0,0.0,1.086906,0.023184,0.106446,-0.02,-0.004,-15.000000,0.000000,0.0,0.000000,-5.00,-1.000,-7.100000,0.000000,0.0,0.000000
10401,2024-01-23,Polônia 1. Liga,14:00,MKKS Koszalin,Slask Wroclaw II,1.71,2.0

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Sacramento Kings,Atlanta Hawks,236.5,1.83,0.9999,Over
1,21:00,Eua Nba,Indiana Pacers,Denver Nuggets,240.5,1.89,1.0000,Over
